# 🔧 Run All Setup Tasks

This notebook automates the common steps:
1. Install dependencies (if running in fresh env)
2. Check `.env`
3. Run BigQuery setup
4. Run Vertex AI Search export/setup
5. Run pytest suite

⚠️ Requires: `.env` populated and `gcloud auth application-default login` done once.

In [ ]:
# --- Make repo imports (cli/, backend/, tools/) work inside this notebook ---
import os, sys, pathlib

# Find the repo root by walking up until we see requirements.txt
here = pathlib.Path().resolve()
candidates = [here, *here.parents]
root = None
for p in candidates:
    if (p / "requirements.txt").exists() and (p / "backend").exists():
        root = p
        break

if root is None:
    raise RuntimeError("Could not locate repo root (missing requirements.txt). Move this notebook into the repo or update the search logic.")

# 1) Put repo root at the front of sys.path so `import cli` works
root_str = str(root)
if root_str not in sys.path:
    sys.path.insert(0, root_str)

# 2) Switch working dir to repo root so relative paths (e.g., tests/, notebooks/, etc.) work
os.chdir(root_str)

print("Repo root:", root_str)
print("Python exe:", sys.executable)
print("sys.path[0]:", sys.path[0])


In [ ]:
# ✅ 1. Install requirements (optional if already installed)
!pip install -r requirements.txt

In [ ]:
# ✅ 2. Check environment values
from cli import check_env
check_env.main()

In [ ]:
# ✅ 3. Run BigQuery setup (dataset + tables)
from cli import setup_bq
setup_bq.main()

In [ ]:
# ✅ 4. Run Vertex AI Search setup (bucket + export JSONL)
from cli import setup_search_infra
setup_search_infra.main()

In [ ]:
# ✅ 5. Run schema setup (schema in BQ -> schema in Search)
from cli import setup_search_ingest
setup_search_ingest.main()

In [ ]:
# ✅ 5. Run tests to verify everything is working
!python -m pytest -q